In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.functional as F
from PIL import Image
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization
#from tensorflow.keras.callbacks import EarlyStopping


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')

In [ ]:
df.head()

In [ ]:
len(df.pixels)

In [ ]:
X=np.zeros(shape=(23705,2304))

for i in range(len(df.pixels)):
    df.pixels[i]=np.array(df.pixels[i].split(),dtype='float32')
    X[i]=df.pixels[i]
    
    
    

In [ ]:
X.shape # my image pixels were stored  in this array

In [ ]:
X_reshaped=X.reshape(-1,48,48,1)

In [ ]:
X_reshaped.shape

In [ ]:
X_reshaped[0].shape

In [ ]:
#print random images

index=np.random.randint(0,23704,25)


plt.figure(figsize=(16,16))

for i in range(len(index)):
    plt.subplot(5,5,(i%25)+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_reshaped[index[i]].reshape(48,48))
    plt.xlabel(
        "Age:"+str(df['age'].iloc[index[i]])+
        " Ethnicity:"+str(df['ethnicity'].iloc[index[i]])+
        " Gender:"+str(df['gender'].iloc[index[i]])
    )
    
plt.show()

* Male==0
* Female==1

According to pictures of people(I dont have much idea what this numbers refer to which ethnics)

* 0==White
* 1==Black
* 2==Asian
* 3==İndian
* 4==Latin


In [ ]:
plt.figure(figsize=(40,5))

sns.countplot(x='age',hue='gender',data=df)
plt.xlabel('Age')
plt.title('Age of people according to their gender')

In [ ]:
plt.figure(figsize=(20,5))

sns.countplot(x='ethnicity',hue='gender',data=df)
plt.xlabel('Ethnicity')
plt.title('Ethnicity of people according to their gender')

In [ ]:
plt.figure(figsize=(20,5))

sns.countplot(x='ethnicity',data=df)
plt.xlabel('Ethnicity')
plt.title('Ethnicity of people') # Unbalanced

In [ ]:
plt.figure(figsize=(20,5))

sns.countplot(x='age',data=df)
plt.xlabel('Age')
plt.title('Age of people') 

In [ ]:
plt.figure(figsize=(20,5))

sns.countplot(x='gender',data=df)
plt.xlabel('Gender')
plt.title('Gender of people') 

In [ ]:
df.gender=df.gender.astype('object')
df.ethnicity=df.ethnicity.astype('object')


In [ ]:
# Labels for images
from keras.utils import to_categorical

Y_age=np.array([df.age]).T
Y_Gender=np.array(to_categorical(df.gender))
Y_Ethnic=np.array(to_categorical(df.ethnicity))

In [ ]:
from sklearn.preprocessing import LabelEncoder

le =LabelEncoder()

Y_Gender2=le.fit_transform(df.gender)

In [ ]:
print("Shape of Age Labels:",Y_age.shape)
print("Shape of Gender Labels:",Y_Gender.shape)
print("Shape of Ethnicity Labels:",Y_Ethnic.shape)

In [ ]:
Y_Gender[:5] #male==[1,0],female==[0,1]

In [ ]:
Y_Ethnic[:5]

## Gender Prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X_reshaped, Y_Gender, test_size=0.3, random_state=42,shuffle=True)

In [ ]:
print('Shape of X_train:',X_train.shape)
print('Shape of X_test:',X_test.shape)
print('Shape of y_train:',y_train.shape)
print('Shape of y_test:',y_test.shape)

In [ ]:
X_test, X_for_pred, y_test, y_for_pred = train_test_split(X_test, y_test, test_size=0.1, random_state=42,shuffle=True)

In [ ]:
print('Shape of X_test:',X_test.shape) # I use this for my model training
print('Shape of X_for_pred:',X_for_pred.shape) # I this them for prediction 
print('Shape of y_test:',y_test.shape) #I use this for my model training
print('Shape of y_for_pred:',y_for_pred.shape) # I this them for prediction 

In [ ]:
X_train=X_train/255.0
X_test=X_test/255.0
X_for_pred=X_for_pred/255.0

## Building Model(Keras)

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(48,48,1),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(256, activation='relu')),
model.add(BatchNormalization()),
model.add(Dropout(0.5)),
model.add(Dense(2, activation='sigmoid'))

model.summary()
          




In [ ]:
batch_size=128
epochs=12

from tensorflow.keras.optimizers import Adam

opt=Adam(lr=0.001)

model.compile(optimizer = opt,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,batch_size=batch_size,epochs = epochs, validation_data = (X_test,y_test))

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(X_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

In [ ]:
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
y_pred=model.predict(X_for_pred)
y_pred[:5]

In [ ]:
Y_pred_classes = np.argmax(y_pred,axis = 1)  # convert predictions to one hot vectors

In [ ]:
Y_pred=['male' if i==0 else 'female' for i in Y_pred_classes]

In [ ]:
Y_pred[:5]

In [ ]:
index=np.random.randint(0,712,25) # predicted label and their images

plt.figure(figsize=(16,16))

for i in range(len(index)):
    plt.subplot(5,5,(i%25)+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_for_pred[index[i]].reshape(48,48))
    plt.xlabel("Predicted Gender:"+str(Y_pred[index[i]])
    )
    
plt.show()

In [ ]:
#Confusion Matrix

Y_true = np.argmax(y_for_pred,axis = 1) 

# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

70 false prediction

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_true, Y_pred_classes)) 

## Gender Classification(Pytorch)

In [ ]:
X.shape #I use this for Pytorch

In [ ]:
X_new=X.reshape(23705,1,48,48)

In [ ]:
Y_Gender2.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_Gender2, test_size=0.3, random_state=42,shuffle=True)
X_test, X_for_pred, y_test, y_for_pred = train_test_split(X_test, y_test, test_size=0.1, random_state=42,shuffle=True)

In [ ]:
x_train_tensor=torch.from_numpy(X_train)
print("x_train tensor size:",x_train_tensor.size())

y_train_label=torch.from_numpy(y_train)
print("y_train_label size:",y_train_label.size())


In [ ]:
x_test_tensor=torch.from_numpy(X_test)
print("x_test tensor size:",x_test_tensor.size())

y_test_label=torch.from_numpy(y_test)
print("y_test_ label size:",y_test_label.size())

In [ ]:
x_for_pred_tensor=torch.from_numpy(X_for_pred)
print("x_test tensor size:",x_for_pred_tensor.size())

y_for_pred_label=torch.from_numpy(y_for_pred)
print("y_test_ label size:",y_for_pred_label.size())

In [ ]:
batch_size = 100
n_iters = 200
num_epochs = n_iters / (len(X_test) / batch_size)
num_epochs = int(num_epochs)

In [ ]:
num_epochs

In [ ]:
train=torch.utils.data.TensorDataset(x_train_tensor,y_train_label) # x ve y train birleştirildi
trainloader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True) # data ya çeviriyoruz eğitime uygun hale geldi

    
test=torch.utils.data.TensorDataset(x_test_tensor,y_test_label) # x ve y train birleştirildi
testloader=torch.utils.data.DataLoader(test,batch_size=batch_size,shuffle=False)

for_pred=torch.utils.data.TensorDataset(x_for_pred_tensor,y_for_pred_label)
for_pred_loader=torch.utils.data.DataLoader(for_pred,batch_size=batch_size,shuffle=False)

## Building Model

In [ ]:


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # Max pool 1
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
        self.Batch1=nn.BatchNorm2d(16)
        self.Batch2=nn.BatchNorm2d(32)
        self.Batch3=nn.BatchNorm2d(64)
        self.Batch4=nn.BatchNorm2d(128)
        
        self.Drop1=nn.Dropout(0.2)
        self.Drop2=nn.Dropout(0.5)


        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.cnn4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        
        

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(128 * 1 * 1, 128) 
        self.fc2=nn.Linear(128,256)
        self.fc3=nn.Linear(256,2)

    def forward(self, x):
 
        out = self.cnn1(x) 
        out = self.relu(out)
        out = self.maxpool(out)
        out=self.Batch1(out)
        out=self.Drop1(out)
 
        out = self.cnn2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out=self.Batch2(out)
        out=self.Drop1(out)
        
        out = self.cnn3(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out=self.Batch3(out)
        out=self.Drop1(out)
        
        out = self.cnn4(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out=self.Batch4(out)
        out=self.Drop1(out)
        

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        out=self.Drop2(out)
        
        out=self.fc2(out)
        
        out=self.Drop2(out)
        
        out=self.fc3(out)
        
        

        return out

In [ ]:
model = CNNModel()

learning_rate=0.001

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
print(model.parameters())

print(len(list(model.parameters())))

# Convolution 1: 16 Kernels
print(list(model.parameters())[0].size())

# Convolution 1 Bias: 16 Kernels
print(list(model.parameters())[1].size())

# Convolution 2: 32 Kernels with depth = 16
print(list(model.parameters())[2].size())

# Convolution 2 Bias: 32 Kernels with depth = 16
print(list(model.parameters())[3].size())

# Fully Connected Layer 1
print(list(model.parameters())[4].size())

# Fully Connected Layer Bias
print(list(model.parameters())[5].size())

In [ ]:
loss_list=[]
accuracy_list=[]
iter_list=[]

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Load images
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images.float())

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 10 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                # Load images
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images.float())

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct // total

            # Print Loss
            accuracy_list.append(accuracy)
            loss_list.append(loss.item())
            iter_list.append(iter)
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [ ]:
%matplotlib inline
acc = accuracy_list
loss = loss_list
iteration=iter_list

plt.plot(iteration, acc, 'r', label='Model accuracy')
plt.plot(iteration, loss, 'b', label='Model Loss')
plt.title('Accuracy and Loss')
plt.legend()
plt.figure()



In [ ]:
plt.plot(iteration, acc, 'r', label='Model accuracy')
#plt.plot(iteration, loss, 'b', label='Model Loss')
plt.title('Accuracy and Loss')
plt.legend()
plt.figure()

In [ ]:
#plt.plot(iteration, acc, 'r', label='Model accuracy')
plt.plot(iteration, loss, 'b', label='Model Loss')
plt.title('Accuracy and Loss')
plt.legend()
plt.figure()

I will use y_for_pred data for rediction later please upvote